# Демонстрация работы классификатора

Сейчас мы находимся в папке <code>./source<code>, все данные находятся в папке <code>./data<code>. Помимо прочего, там содержатся txt-файлы распарсенных статей и файл с отметками некоторых статей. Перед тем, как обучать на них классификатор, полезно превратить эти статьи в мешки слов, причем постараться избавиться от словоформ.

За это отвественнен модуль <code>normalizing<code>. Он берет множество распарсенных статей, множество статей с отметками и "нормализует" их, то есть превращает в мешок слов те статьи, которые еще не нормализованы.

У normalizing есть два режима, в которых он по-разному составляет мешки слов. По умолчанию это режим, когда просто ищутся все вхождения слов регуляркой <code>'\w+[\-\w+]*'<code> и потом с помощью pymorphy2 отсеиваются служебные части речи: местоимения, предлоги, союзы, частицы, междометия. Оставшиеся слова приводятся к нормальной форме с помощью той же библиотеки и записываются в мешок слов.

Второй режим можно включить ключом --empirical. У него более сложные регулярки и чуть более сложная система проверок, которые просто показались мне логичными, но не имеют никакого серьезного обоснования (отсюда и название). Сразу отсекаются русские слова короче 4 символов и английские короче 2. Слова вроде 'android-гаджет' раскладываются в два слова. Оставшиеся слова аналогично приводятся к нормальной форме и заносятся в мешок слов.

Помимо прочего, мы отдельно нормализуем и сохраняем в разные файлы заголовки и сам текст статьи. В дальнейшем мы будет на них независимо обучаться.

Для начала отчистим старые мешки слов.

In [1]:
! python3 normalizing.py --clean

Теперь создадим новые мешки слов (дефолтный режим).

In [2]:
! python3 normalizing.py

Для примера, вот так выглядит мешок слов для статьи <link>https://geektimes.ru/company/payonline/blog/272648/<link>:

In [3]:
!cat ../data/normalized/272648.txt

автомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineавтомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineавтомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineавтомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineавтомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineконец
февраль
visa
представить
необычный
платёжный
решение
приложение
который
помочь
водитель
оплачивать
бензин
заправка
выходить
машина
visa
honda
объединиться
воплотить
жизнь
новое
платёжный
приложение
автомобиль
презентация
который
состояться
mobile
world
congress
барселона
позволять
оплачивать
парковка
заправка
автомобиль
выходить
план
разработка
подобный
приложение
автомобиль
visa
заявить
год
назад
visa
honda
создать
приложение
который
пользоваться
непосредственно
панель
управление
автомобиль
сообщать
водитель
низок
уровень
топливо
строить
маршрут
ближний
заправка
машина
парковаться
возле
топливный
насос
приложение
рассчитывать
стоимость
заправка
предл

Теперь приступим к собственно обучению.

In [5]:
from classifier import *


Мы будем использовать три классификатора и сравнивать их работу. Введем их: создадим для каждого свой pipeline и парметры для grid search-а.

In [6]:
sgd, multi_nb, bernoulli_nb = ClassifierData(), ClassifierData(), ClassifierData()
sgd.name = 'SGDClassifier'
multi_nb.name = 'MultinomialNB'
bernoulli_nb.name = 'BernoulliNB'
sgd.clf = Pipeline([('cnt', TfidfVectorizer()),
                    ('clf', SGDClassifier(loss='log'))
                    ])
multi_nb.clf = Pipeline([('cnt', TfidfVectorizer()),
                         ('clf', MultinomialNB())
                        ])
bernoulli_nb.clf = Pipeline([('cnt', CountVectorizer()),
                             ('clf', BernoulliNB(binarize=0.0))
                            ])
sgd.parameters = {'clf__penalty': ['l2', 'l1', 'elasticnet'],
                  'clf__l1_ratio': [0.0, 0.01, 0.05, 0.10, 0.2, 0.3, 0.4, 0.5],
                  'clf__alpha': [0.001, 0.0001, 0.00001, 0.000001, 0.0000001],
                  'cnt__use_idf': (True, False),
                 }
multi_nb.parameters = {'clf__alpha': [0.001, 0.0001, 0.00001, 0.000001, 0.0000001],
                       'cnt__use_idf': (True, False),
                      }
bernoulli_nb.parameters = {'clf__alpha': [0.001, 0.0001, 0.00001, 0.000001, 0.0000001],
                          }
all_classifiers = [sgd, multi_nb, bernoulli_nb]

Создадим тестовую и тренировочную выборки:

In [7]:
full_struct = get_a_structures(test_part=0.2)
train, test = full_struct.train, full_struct.test
print(len(train.data), len(test.data))

159 41


Для тестовой выборки отбиралось по 0.2 от всех элементов каждого класса. Если посмотреть на цифры, то будет следующее:

In [8]:
print("train positive:", train.target.count(1))
print("train negative:", train.target.count(-1))
print("test positive:", test.target.count(1))
print("test negative:", test.target.count(-1))

train positive: 68
train negative: 91
test positive: 18
test negative: 23


"Плохих" статей несколько больше. Но что уж тут поделать.

Переходя к обучению, рассмотрим следующую функцию:

In [9]:
def choose_the_classifier(classifiers_to_check, data, target):
    best_classifier = None
    best_score = 0;
    best_name = ''
    for cur_classifier in classifiers_to_check:
        log.debug('Now training ' + cur_classifier.name)
        searcher = GridSearchCV(estimator=cur_classifier.clf,
                                param_grid=cur_classifier.parameters,
                                scoring=scorer,
                                cv=3,
                                n_jobs=-1
                                )
        t = time()
        searcher.fit(data, target)
        t = time() - t
        log.debug("{} min, {} sec".format(int(t // 60), round(t % 60, 3)))
        cur_best_score = searcher.best_score_
        log.debug("best score (auc): {}".format(round(cur_best_score, 3)))
        log.debug("best parameters: {}".format(searcher.best_params_))
        if cur_best_score > best_score:
            best_score = cur_best_score
            best_classifier = searcher.best_estimator_
            best_name = cur_classifier.name
    return BestClassifier(clf=best_classifier, name=best_name)

Она принимает на вход лист классификаторов (те самые, которые мы вводили в самом начале) и тренировочную выборку, data и target.

Для каждого классификатора создается свой Grid Search (в качестве scorer выбран auc) с ранее определенными параметрами и, собственно, обучается. Среди всех классификаторов выбирается тот, который показал наилучший результат.

Соответственно, эту функцию мы сейчас будем использовать. Для начала, обучимся на заголовках статей:

In [10]:
print("Training... ", end ='')
t = time()
best_titles_clf = choose_the_classifier(all_classifiers, train.titles, train.target) 
print(round(time() - t, 3), "sec")
print("best for titles:", best_titles_clf.name)

Training... 4.651 sec
best for titles: SGDClassifier


Заголовки штуки короткие, так что обучение происходит быстро.

Перед тем как переходить к обучению на непосредественно текстах статей, добавим в параметры каждого классификатора еще несколько полей:

In [11]:
for clf in all_classifiers:
    clf.parameters['cnt__max_df'] = [0.85, 0.9, 0.95, 1.0]
    clf.parameters['cnt__min_df'] = [0.01, 0.05, 0.10, 0.15, 1]

Как уже говорилось, заголовки короткие, поэтому попытка ограничивать их по df почти всегда приводит к пустой выборке. Поэтому эти параметры мы добавляем только сейчас.

И вот, начинаем обучать классификаторы для текстов. Они длиннее, количество параметров для сетки увеличилось, так что это займет уже на порядок больше времени.

In [12]:
print("Training... ", end ='')
t = time()
best_texts_clf = choose_the_classifier(all_classifiers, train.data, train.target) 
t = time() - t
print("{} min, {} sec".format(int(t // 60), round(t % 60, 3)))
print("best for text:", best_texts_clf.name)

Training... 17 min, 35.89 sec
best for text: MultinomialNB


Теперь самое время проверить результат на тестовой выборке!

In [13]:
pred_titles = best_titles_clf.clf.predict_proba(test.titles)
pred_texts = best_texts_clf.clf.predict_proba(test.data)
predicted_proba = list()
predicted_target = list()
for i in range(len(test.data)):
    prob_neg = 0.5 * (pred_titles[i][0] + pred_texts[i][0])
    prob_pos = 0.5 * (pred_titles[i][1] + pred_texts[i][1])
    predicted_proba.append(prob_pos)
    if prob_neg > prob_pos:
        predicted_target.append(-1)
    else:
        predicted_target.append(1)


print("auc:", round(metrics.roc_auc_score(test.target, predicted_proba), 3))
print("confusion matrix:\n", metrics.confusion_matrix(test.target, predicted_target))
print(metrics.classification_report(test.target, predicted_target,
                                    target_names=test.target_names))

auc: 0.691
confusion matrix:
 [[17  6]
 [10  8]]
             precision    recall  f1-score   support

   Negative       0.63      0.74      0.68        23
   Positive       0.57      0.44      0.50        18

avg / total       0.60      0.61      0.60        41



Вот такой резульатат на тестовой выборке мы получаем. В целом, довольно неплохо.

Теперь вернемся в самое начало и вспомним, что у нас есть еще один способ составления мешка слов. Проверим, какой результат будет для него.

In [14]:
! python3 normalizing.py --clean
! python3 normalizing.py --empirical

Посмотрим на том же примере, как выглядит мешок слов:

In [17]:
! cat ../data/normalized/272648.txt

автомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineавтомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineавтомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineавтомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineавтомобиль
оплата
nfc
автоматизация
технология
visa
honda
payonlineконец
февраль
visa
представить
необычный
платёжный
решение
приложение
который
помочь
водитель
оплачивать
бензин
заправка
выходить
машина
visa
honda
объединиться
чтобы
воплотить
жизнь
новое
платёжный
приложение
автомобиль
презентация
который
состояться
mobile
world
congress
барселона
позволять
оплачивать
парковка
заправка
автомобиль
выходить
план
разработка
подобный
приложение
автомобиль
visa
заявить
назад
visa
honda
создать
приложение
который
можно
пользоваться
непосредственно
панель
управление
автомобиль
сообщать
водитель
низок
уровень
топливо
строить
маршрут
ближний
заправка
когда
машина
парковаться
возле
топливный
насос
приложение
рассчитывать
стоимость


Теперь повторяем все предыдущие действия. Но не забываем для заголовков настроить границы df на дефолтные, в противном случае вылетим с ошибкой.

In [16]:
full_struct = get_a_structures(test_part=0.2)
train, test = full_struct.train, full_struct.test

print("Training with titles... ", end ='')
for clf in all_classifiers:
    clf.parameters['cnt__max_df'] = [1.0]
    clf.parameters['cnt__min_df'] = [1]
t = time()
best_titles_clf = choose_the_classifier(all_classifiers, train.titles, train.target) 
print(round(time() - t, 3), "sec")
print("best for titles:", best_titles_clf.name)

print("Training with texts... ", end ='')
for clf in all_classifiers:
    clf.parameters['cnt__max_df'] = [0.85, 0.9, 0.95, 1.0]
    clf.parameters['cnt__min_df'] = [0.01, 0.05, 0.10, 0.15, 1]
t = time()
best_texts_clf = choose_the_classifier(all_classifiers, train.data, train.target) 
t = time() - t
print("{} min, {} sec".format(int(t // 60), round(t % 60, 3)))
print("best for texts:", best_texts_clf.name)

pred_titles = best_titles_clf.clf.predict_proba(test.titles)
pred_texts = best_texts_clf.clf.predict_proba(test.data)
predicted_proba = list()
predicted_target = list()
for i in range(len(test.data)):
    prob_neg = 0.5 * (pred_titles[i][0] + pred_texts[i][0])
    prob_pos = 0.5 * (pred_titles[i][1] + pred_texts[i][1])
    predicted_proba.append(prob_pos)
    if prob_neg > prob_pos:
        predicted_target.append(-1)
    else:
        predicted_target.append(1)


print("auc:", round(metrics.roc_auc_score(test.target, predicted_proba), 3))
print("confusion matrix:\n", metrics.confusion_matrix(test.target, predicted_target))
print(metrics.classification_report(test.target, predicted_target,
                                    target_names=test.target_names))

Training with titles... 4.578 sec
best for titles: SGDClassifier
Training with texts... 17 min, 24.524 sec
best for texts: MultinomialNB
auc: 0.664
confusion matrix:
 [[18  5]
 [ 8 10]]
             precision    recall  f1-score   support

   Negative       0.69      0.78      0.73        23
   Positive       0.67      0.56      0.61        18

avg / total       0.68      0.68      0.68        41



Этот метод показал более плохой результат на тестовой выборке, если смотреть на auc, но более хороший, если смотреть accuracy.

Любопытно, что оба метода выбрали одинаковые классификаторы для заголовков и одинаковые для текстов.

Вообще, главная моя проблема --- это очень маленькая выборка. В зависимости от того, как исходно были перемешанные размеченные данные и с чего в итоге состоят тестовая и тренировочная выборка, может сильно зависеть результат.

Что посмотреть, что происходит, можно просто несколько раз запустить программу.

In [18]:
! python3 classifier.py --steps=3

==STEP 1==
Training... 17 min, 55.168 sec

PREDICTED QUALITY
best for titles: SGDClassifier
best for text: MultinomialNB
auc: 0.705
[[14  9]
 [ 8 10]]
             precision    recall  f1-score   support

   Negative       0.64      0.61      0.62        23
   Positive       0.53      0.56      0.54        18

avg / total       0.59      0.59      0.59        41

==STEP 2==
Training... 17 min, 53.75 sec

PREDICTED QUALITY
best for titles: SGDClassifier
best for text: MultinomialNB
auc: 0.727
[[17  6]
 [ 7 11]]
             precision    recall  f1-score   support

   Negative       0.71      0.74      0.72        23
   Positive       0.65      0.61      0.63        18

avg / total       0.68      0.68      0.68        41

==STEP 3==
Training... 15 min, 35.794 sec

PREDICTED QUALITY
best for titles: SGDClassifier
best for text: SGDClassifier
auc: 0.725
[[14  9]
 [ 8 10]]
             precision    recall  f1-score   support

   Negative       0.64      0.61      0.62        23
   Positive

По сути говоря, мы просто три раза запускаем программу. Каждое новое обучение происходит совершенно независимо от предыдущего. Цель данной демонстрации --- показать, какая стабильность у полученного результата.

Сейчас, напомню, был "эмпирический" режим составления мешков слов. Посмотрим теперь, что будет в "дефолтном" режиме.

In [1]:
! python3 normalizing.py --clean
! python3 normalizing.py

In [2]:
! python3 classifier.py --steps=3

==STEP 1==
Training... 17 min, 17.395 sec

PREDICTED QUALITY
best for titles: SGDClassifier
best for text: MultinomialNB
auc: 0.688
[[17  6]
 [ 8 10]]
             precision    recall  f1-score   support

   Negative       0.68      0.74      0.71        23
   Positive       0.62      0.56      0.59        18

avg / total       0.66      0.66      0.66        41

==STEP 2==
Training... 17 min, 38.743 sec

PREDICTED QUALITY
best for titles: SGDClassifier
best for text: MultinomialNB
auc: 0.713
[[18  5]
 [11  7]]
             precision    recall  f1-score   support

   Negative       0.62      0.78      0.69        23
   Positive       0.58      0.39      0.47        18

avg / total       0.60      0.61      0.59        41

==STEP 3==
Training... 18 min, 3.167 sec

PREDICTED QUALITY
best for titles: SGDClassifier
best for text: SGDClassifier
auc: 0.729
[[19  4]
 [11  7]]
             precision    recall  f1-score   support

   Negative       0.63      0.83      0.72        23
   Positive

Видим, что разброс не очень сильный (но скорее всго это потому, что не успела пронать большое количество раз). Но если мы посмотрим, какие результаты получилась в первых демонстрациях, то увидим, что там они были все-таки несколько ниже -- то есть тут уже можно заметить эффект маленькой выборки.